## LSTM

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

Embeding

In [5]:
max_features = 20000
maxlen = 20
embedding_size = 128

LSTM

In [6]:
lstm_output_size = 128

Training

In [7]:
batch_size = 32
epochs = 20

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [9]:
def load_ds(path):
    vecfile = open(path,'r')
    return pickle.load(vecfile)

In [10]:
ds = load_ds('./data/ann/vectors_50_50.txt')
ds.head()

category                                          tweet_sec
0         1                      [268, 459, 146, 2, 470, 4493]
1         1                              [2264, 7, 9784, 3050]
2         0  [80, 172, 35, 1, 14, 39, 2265, 7, 9785, 2266, ...
3         1  [107, 41, 21, 471, 9, 659, 5, 216, 300, 5, 7, ...
4         1                       [562, 106, 29, 563, 29, 422]

In [14]:
print('Pad sequences (samples x time)')
tweet_sec = sequence.pad_sequences(ds['tweet_sec'], maxlen=maxlen)
print('tweet shape:', tweet_sec.shape)

Pad sequences (samples x time)
('tweet shape:', (15306, 20))


In [11]:
def build_arch():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, embedding_size))
    model.add(LSTM(lstm_output_size,
                   activation='tanh', 
                   recurrent_activation='hard_sigmoid', 
                   dropout=0.2, 
                   recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [15]:
def cross_val(tweet_sec, target, nFolds):
    kFold = StratifiedKFold(n_splits=nFolds, shuffle=True)
    scores = []
    for train, test in kFold.split(tweet_sec,target):
        model = build_arch()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(tweet_sec[train],target[train],batch_size=batch_size,epochs=epochs,verbose=1)
        score, acc = model.evaluate(tweet_sec[test], target[test],batch_size=batch_size)
        scores.append(acc * 100)
    return scores   

In [12]:
#Training with the 100% of the data set
def train_model(tweet_sec,target):
    model = build_arch()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model.fit(tweet_sec, target,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)

In [17]:
results = cross_val(tweet_sec,ds['category'],10)

Build model...
Epoch 1/20
13774/13774 [==============================] - 30s - loss: 0.3869 - acc: 0.8298    
Epoch 2/20
13774/13774 [==============================] - 30s - loss: 0.1897 - acc: 0.9299    
Epoch 3/20
13774/13774 [==============================] - 30s - loss: 0.1144 - acc: 0.9594    
Epoch 4/20
13774/13774 [==============================] - 30s - loss: 0.0763 - acc: 0.9731    
Epoch 5/20
13774/13774 [==============================] - 30s - loss: 0.0536 - acc: 0.9808    
Epoch 6/20
13774/13774 [==============================] - 30s - loss: 0.0443 - acc: 0.9845    
Epoch 7/20
13774/13774 [==============================] - 30s - loss: 0.0316 - acc: 0.9890    
Epoch 8/20
13774/13774 [==============================] - 30s - loss: 0.0292 - acc: 0.9892    
Epoch 9/20
13774/13774 [==============================] - 30s - loss: 0.0261 - acc: 0.9917    
Epoch 10/20
13774/13774 [==============================] - 30s - loss: 0.0179 - acc: 0.9940    
Epoch 11/20
13774/13774 [=========

13776/13776 [==============================] - 29s - loss: 0.0493 - acc: 0.9820    - ETA: 2s -  - ETA: 0s - loss: 0.0494 - acc: 0.
Epoch 6/20
13776/13776 [==============================] - 29s - loss: 0.0380 - acc: 0.9880    
Epoch 7/20
13776/13776 [==============================] - 29s - loss: 0.0282 - acc: 0.9906    
Epoch 8/20
13776/13776 [==============================] - 29s - loss: 0.0215 - acc: 0.9940    - ETA: 1s - loss: 0.0218 - acc: 0.9 - ETA: 1s - loss: 0.
Epoch 9/20
13776/13776 [==============================] - 29s - loss: 0.0225 - acc: 0.9917    
Epoch 10/20
13776/13776 [==============================] - 30s - loss: 0.0135 - acc: 0.9964    
Epoch 11/20
13776/13776 [==============================] - 30s - loss: 0.0203 - acc: 0.9935    
Epoch 12/20
13776/13776 [==============================] - 30s - loss: 0.0130 - acc: 0.9959    
Epoch 13/20
13776/13776 [==============================] - 31s - loss: 0.0129 - acc: 0.9949    
Epoch 14/20
13776/13776 [========================

In [27]:
print("Acc: %.2f (+/- %.2f)" %(np.mean(results), np.std(results)))

Acc: 86.19 (+/- 0.91)


In [34]:
for i in range (0,len(results)):
    print "Model %d, acc: %.2f " %(i+1,results[i])

Model 1, acc: 86.68 
Model 2, acc: 85.64 
Model 3, acc: 87.21 
Model 4, acc: 84.84 
Model 5, acc: 86.54 
Model 6, acc: 84.44 
Model 7, acc: 87.32 
Model 8, acc: 86.54 
Model 9, acc: 86.73 
Model 10, acc: 86.01 


In [16]:
trained_model = train_model(tweet_sec,ds['category'])

Build model...
Epoch 1/20
15306/15306 [==============================] - 35s - loss: 0.3761 - acc: 0.8348    
Epoch 2/20
15306/15306 [==============================] - 34s - loss: 0.1722 - acc: 0.9369    
Epoch 3/20
15306/15306 [==============================] - 34s - loss: 0.0984 - acc: 0.9637    
Epoch 4/20
15306/15306 [==============================] - 34s - loss: 0.0611 - acc: 0.9789    
Epoch 5/20
15306/15306 [==============================] - 34s - loss: 0.0439 - acc: 0.9856    
Epoch 6/20
15306/15306 [==============================] - 34s - loss: 0.0353 - acc: 0.9880    
Epoch 7/20
15306/15306 [==============================] - 34s - loss: 0.0232 - acc: 0.9924    
Epoch 8/20
15306/15306 [==============================] - 34s - loss: 0.0228 - acc: 0.9926    
Epoch 9/20
15306/15306 [==============================] - 34s - loss: 0.0194 - acc: 0.9933    
Epoch 10/20
15306/15306 [==============================] - 34s - loss: 0.0164 - acc: 0.9939    
Epoch 11/20
15306/15306 [=========

In [17]:
model_json = trained_model.model.to_json()
with open("./data/models/lstm.json",'w') as json_file:
    json_file.write(model_json)
trained_model.model.save_weights("./data/models/lstm_weights.h5")

In [18]:
trained_model.model.save("./data/models/lstm_complete.h5")